In [ ]:
import time
import cv2
import handtrackingModule2 as htm
import os
import numpy as np
import subprocess
import math
from keras.models import load_model

def set_brightness(bright):
    if 0 <= bright <= 1:
        subprocess.run(["brightness", str(bright)])

def hand_to_bright(length, min_len=50, max_len=500):
    min_light = 0
    max_light = 1
    bright = np.interp(length, [min_len, max_len], [min_light, max_light])
    return bright

def set_volume(volume):
    if 0 <= volume <= 100:
        subprocess.run(["osascript", "-e", f"set volume output volume {volume}"])
        print(volume)
    else:
        print("Volume must be within 1 to 100")

def hand_to_volume(length, min_len=50, max_len=500):
    min_vol = 0
    max_vol = 100
    volume = np.interp(length, [min_len, max_len], [min_vol, max_vol])
    return volume

brushThickness = 50
eraserThickness = 50
xp, yp = 0, 0

folderpath = "/Users/chandrakant/Desktop/virtual"
myList = os.listdir(folderpath)
myList.pop(0)
print(myList)

overlayList = [cv2.imread(f'{folderpath}/{imPath}') for imPath in myList]
drawcolour = (0, 0, 0)

header = overlayList[8]

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)
imgCanvas = np.zeros((720, 1280, 3), np.uint8)
detector = htm.handDetector(detectionCon=0.85)

# Load models once
model1 = load_model('my_model1.keras')
model2 = load_model('my_model2.keras')
model3 = load_model('my_model.keras')

def preprocess_image(img):
    img = cv2.resize(img, (28, 28))
    img = img.astype('float32') / 255.0
    img = img.reshape(1, 28, 28, 1)
    return img

prediction_made = False
prediction_made1 = False
prediction_made2 = False
a=0
b=0

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img = detector.findHands(img)
    lmlist = detector.findPosition(img, draw=False)

    if len(lmlist) != 0:
        x1, y1 = lmlist[8][1:]
        x2, y2 = lmlist[12][1:]
        fingers = detector.fingersUp()
        a=x1
        b=y1

        if fingers[1] and fingers[2]:
            xp, yp = 0, 0
            if y1 < 78:
                if 660 < x1 < 700:
                    header = overlayList[6]
                    drawcolour = (255, 255, 255)
                elif 580 < x1 < 640:
                    header = overlayList[7]
                    drawcolour = (0, 0, 255)
                elif 520 < x1 < 560:
                    header = overlayList[2]
                    drawcolour = (255, 0, 0)
                elif 460 < x1 < 500:
                    header = overlayList[3]
                    drawcolour = (0, 255, 0)
                elif 370 < x1 < 420:
                    header = overlayList[5]
                    drawcolour = (0, 0, 0)
                elif 210 < x1 < 340:
                    header = overlayList[4]
                elif 110 < x1 < 150:
                    header = overlayList[0]
                elif 10 < x1 < 40:
                    header = overlayList[1]
                else:
                    header = overlayList[8]
            cv2.rectangle(img, (x1, y1 - 15), (x2, y2 + 15), drawcolour, cv2.FILLED)

        if fingers[1] and not fingers[2]:
            cv2.circle(img, (x1, y1), 15, drawcolour, cv2.FILLED)
            if xp == 0 and yp == 0:
                xp, yp = x1, y1
            if drawcolour == (0, 0, 0):
                cv2.line(img, (xp, yp), (x1, y1), drawcolour, eraserThickness)
                cv2.line(imgCanvas, (xp, yp), (x1, y1), drawcolour, eraserThickness)
            else:
                cv2.line(img, (xp, yp), (x1, y1), drawcolour, brushThickness)
                cv2.line(imgCanvas, (xp, yp), (x1, y1), drawcolour, brushThickness)
            xp, yp = x1, y1

    imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)
    _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)
    imgInv = cv2.cvtColor(imgInv, cv2.COLOR_GRAY2BGR)
    img = cv2.bitwise_and(img, imgInv)
    img = cv2.bitwise_or(img, imgCanvas)

    img[0:78, 0:720] = header
    img = cv2.addWeighted(img, 0.5, imgCanvas, 0.5, 0)

    cv2.imshow("Image", img)
    cv2.imshow("Canvas", imgCanvas)

    if b < 78 and 210 < a < 340 and not prediction_made:
        prediction_made = True
        header = overlayList[4]
        img[0:78, 0:720] = header
        img_g = preprocess_image(imgGray)
        prediction1 = model1.predict(img_g)
        prediction2 = model2.predict(img_g)
        prediction3 = model3.predict(img_g)

        from collections import Counter
        votes = [np.argmax(prediction1), np.argmax(prediction2), np.argmax(prediction3)]
        vote_counts = Counter(votes)
        most_common_value, count = vote_counts.most_common(1)[0]

        print(f"Predicted value: {most_common_value}")
        cv2.putText(img, f'Predicted value: {most_common_value}', (150, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 3)

        cv2.imshow("Image", img)
        cv2.waitKey(1)
        time.sleep(5)
        break

    if b < 78 and 110 < a < 150 and not prediction_made1:
        prediction_made1 = True
        while True:
            success, img = cap.read()
            img = detector.findHands(img)
            lmlist = detector.findPosition(img, draw=False)
            img[0:78, 0:720] = header
            if len(lmlist) != 0:
                x1, y1 = lmlist[4][1], lmlist[4][2]
                x2, y2 = lmlist[8][1], lmlist[8][2]
                cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
                cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
                cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
                cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
                length = math.hypot(x2 - x1, y2 - y1)
                if length < 50:
                    cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)
                    print(length)

            vol = hand_to_volume(length)
            set_volume(vol)
            cv2.imshow("Image", img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    if b < 78 and 10 < a < 40 and not prediction_made2:
        prediction_made2 = True
        while True:
            success, img = cap.read()
            img = detector.findHands(img)
            lmlist = detector.findPosition(img, draw=False)
            cv2.rectangle(img,(100,550),(50,100),(255,0,0),3)
            img[0:78, 0:720] = header
            if len(lmlist) != 0:
                x1, y1 = lmlist[4][1], lmlist[4][2]
                x2, y2 = lmlist[8][1], lmlist[8][2]
                cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
                cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
                cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
                cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
                length = math.hypot(x2 - x1, y2 - y1)
                brightbar = np.interp(length, [50, 500], [550, 100])
                ratio = (int(brightbar) - 100) / (550 - 100)
                color = (0, int(255 * (1 - ratio)), int(255 * ratio))
                cv2.rectangle(img, (100, 550), (50, int(brightbar)), color, cv2.FILLED)
                if length < 50:
                    cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)

                len1 = hand_to_bright(length)
                set_brightness(len1)

            cv2.imshow("Image", img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
